In [21]:
import time
from time import sleep
import traceback
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import  Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import re
from openpyxl import load_workbook
from datetime import date
from glob import glob
from shutil import move
import tabula
from PyPDF2 import PdfReader
mes = '09'
hoje = date.today()
processoSEI = 'SEI-040002/003022/2024'
pasta = r"C:/Users/"+ os.getlogin() +r"/OneDrive - SEFAZ-RJ/Folha EGE/Exercício " + str(hoje.year) + '/' + str(mes) + "." + str(hoje.year) + ' - ' + processoSEI.replace("/", "_")
novaMemoria = pasta + "/" + "Memória de Cálculo - " + str(mes) + "." + str(hoje.year) + ".xlsx"
planilha = load_workbook(novaMemoria)
meses = ["Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho", "Julho", "Agosto", "Setembro", "Outubro", "Novembro", 
         "Dezembro"]
arquivo = pasta + r"\TGRJ0807P_" + meses[int(mes) - 1] +".pdf"
retencoes = planilha["Retenções"]

In [29]:
navegador = webdriver.Firefox()
def loginSIAFE():
    navegador.get("https://siafe2.fazenda.rj.gov.br/Siafe/faces/login.jsp")

    usuario = navegador.find_element(By.XPATH, value='//*[@id="loginBox:itxUsuario::content"]')
    usuario.send_keys(os.environ['cpf'])

    senha = navegador.find_element(By.XPATH, value='//*[@id="loginBox:itxSenhaAtual::content"]')
    senha.send_keys(os.environ['senha_siafe'])
    
    btnLogin = navegador.find_element(By.XPATH, value='//*[@id="loginBox:btnConfirmar"]')
    btnLogin.click()

    try:
        WebDriverWait(navegador,2).until(EC.element_to_be_clickable((By.XPATH, "//a[@class = 'x12k']"))).click()        
    except:
        pass

    navegador.maximize_window()
    
loginSIAFE()
navegador.get(r"https://siafe2.fazenda.rj.gov.br/Siafe/faces/execucao/contabilidade/execucaoContabilidadeMain.jsp")
WebDriverWait(navegador,10).until(EC.element_to_be_clickable((By.XPATH, "//a[text() = 'Detalhamento da Conta Contábil']"))).click()
WebDriverWait(navegador,10).until(EC.element_to_be_clickable((By.XPATH, "//input[contains(@name, 'UnidadeGestora')]"))).send_keys("370200")
navegador.find_element(By.XPATH, '//input[@id = "tplSip:chkSaldoZerado::content"]').click()
select = Select(navegador.find_element(By.XPATH, '//select[@id = "tplSip:cbxMes::content"]'))
select.select_by_index(int(mes))
##CREDITO


TESTE INDIVIDUAL

In [ ]:
def verificarCompetencia():
    navegador.switch_to.default_content()
    WebDriverWait(navegador,20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "ifrArvore")))
    docs = navegador.find_elements(By.XPATH, "//div[@id = 'divArvore']//div//a[@class = 'infraArvoreNo']")
    quantDocs = len(docs) 
    for doc in reversed(range(quantDocs)):
            docTexto = docs[doc].text
            if "Despacho de Encaminhamento de Processo" in docTexto:
                docs[doc].click()
                navegador.switch_to.default_content()            
                WebDriverWait(navegador,20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "ifrVisualizacao")))
                WebDriverWait(navegador,20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "ifrArvoreHtml")))
                WebDriverWait(navegador,5).until(EC.presence_of_element_located((By.XPATH, '//p[contains(text(), "Rio de Janeiro")]')))

                body = navegador.find_element(By.TAG_NAME, 'body').text
                if "processamento do DARJ" in body:
                    if "AUDITORA FISCAL" in body.upper() or "AUDITOR FISCAL" in body.upper():
                        return "ok"
                    else:
                        return "inválida"
                else:
                    navegador.switch_to.default_content()
                    WebDriverWait(navegador,20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "ifrArvore")))
                    docs = navegador.find_elements(By.XPATH, "//div[@id = 'divArvore']//div//a[@class = 'infraArvoreNo']")
            
                        
    return "inválida"

def loginSEI():
    navegador.get("https://sei.rj.gov.br/sip/login.php?sigla_orgao_sistema=ERJ&sigla_sistema=SEI")

    usuario = navegador.find_element(By.XPATH, value='//*[@id="txtUsuario"]')
    usuario.send_keys(os.environ['login_sefaz'])

    senha = navegador.find_element(By.XPATH, value='//*[@id="pwdSenha"]')
    senha.send_keys(os.environ['senha_sefaz'])

    exercicio = Select(navegador.find_element(By.XPATH, value='//*[@id="selOrgao"]'))
    exercicio.select_by_visible_text('SEFAZ')

    btnLogin = navegador.find_element(By.XPATH, value='//*[@id="Acessar"]')
    btnLogin.click()


    navegador.maximize_window()
    time.sleep(5)

    navegador.find_element(By.TAG_NAME, "body").send_keys(Keys.ESCAPE)
    time.sleep(1)
    trocarCoordenacao()

def trocarCoordenacao():
    coordenacao = navegador.find_elements(By.XPATH, "//a[@id = 'lnkInfraUnidade']")[1]
    if coordenacao.get_attribute("innerHTML") == 'SEFAZ/COOEGOE':
        coordenacao.click()
        WebDriverWait(navegador,5).until(EC.presence_of_element_located((By.XPATH, "//div[contains(text(), 'Trocar Unidade')]")))
        navegador.find_element(By.XPATH, "//td[text() = 'SEFAZ/COOAJUR' ]").click() 
        
def abrirPastas():
    navegador.switch_to.default_content()
    WebDriverWait(navegador,20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "ifrArvore")))
    listaDocs = WebDriverWait(navegador,5).until(EC.presence_of_element_located((By.ID, "divArvore")))
    pastas = listaDocs.find_elements(By.XPATH, '//a[contains(@id, "joinPASTA")]')
    for doc in pastas[:-1]:
        doc.click() 
        sleep(4)

In [54]:
navegador = webdriver.Firefox()

processoSEI = "SEI-140001/050926/2024"

loginSEI()

barraPesquisa = navegador.find_element(By.ID, "txtPesquisaRapida")

barraPesquisa.send_keys(processoSEI)
barraPesquisa.send_keys(Keys.ENTER)

abrirPastas()

validade, montante = verificarValorEValidade()

competencia = verificarCompetencia()

print("Validade: " + validade)
print("Montante: " + montante)
print("Competencia: " + competencia)


6.179.624,64
271.597,00


In [132]:
meses = ["Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho", "Julho", "Agosto", "Setembro", "Outubro", "Novembro", "Dezembro"]

from datetime import date
from glob import glob
from shutil import move

def alterarNomeArquivo():
    arquivo = ""
    while not os.path.isfile(arquivo):
        file_list = glob(r"C:\Users\\"+os.getlogin()+r"\Downloads\*.pdf")
        for file in file_list:
            if (int(time.time()) - int(os.stat(file).st_mtime) < 3):
                arquivo = file
                time.sleep(1)
                break
    
    for processo in x:
        if processo in arquivo:
            nome = processo + "_" + meses[date.today() - 1]
            newFile = r"C:\Users\\"+os.getlogin()+r"\Downloads\\" + nome + ".pdf"

    move(arquivo, newFile)
    os.remove(arquivo)

In [5]:

arvore = navegador.find_element(By.ID, "ifrArvore")
visualizacao = navegador.find_element(By.ID, "ifrVisualizacao")
navegador.switch_to.frame(arvore)

listaDocs = navegador.find_element(By.ID, "divArvore")
docs = listaDocs.find_elements(By.TAG_NAME, "a")

for doc in docs:
    if "FOLHA" in doc.text.upper(): 
        doc.click()
        navegador.switch_to.default_content()
        navegador.switch_to.frame(visualizacao)
        navegador.switch_to.frame(navegador.find_element(By.ID, "ifrArvoreHtml"))
        time.sleep(0.3)
        
        
        navegador.find_element(By.XPATH, '//button[@id ="secondaryToolbarToggle" ]').click()
        time.sleep(0.3)

        navegador.find_element(By.XPATH, '//button[@id ="secondaryDownload"]').click()
        time.sleep(2)

        pyautogui.press("enter")
        alterarNomeArquivo()
        navegador.switch_to.default_content()
        navegador.switch_to.frame(arvore)
    


NameError: name 'alterarNomeArquivo' is not defined